# Import txt output from Buehler code

In [ ]:
!pip install xlsxwriter

In [ ]:
import os 
import glob
import pandas as pd

In [2]:
# Specify the directory
current_path = os.getcwd()
directory_path = os.path.join(current_path, 'buehler_output')

# Use glob to find all .txt files in the directory
txt_files = glob.glob(os.path.join(directory_path, '*.txt'))

In [ ]:
identifiers = [
    "0,2", 
    "10,0",
    "25,0"
]
with pd.ExcelWriter(os.path.join(current_path, 'metrics.xlsx'), engine='xlsxwriter') as writer:
    for identifier in identifiers:
        # Initialize dictionaries to store metrics
        metric1_data = {}
        metric2_data = {}
        count = 0
        # Loop through the list of .txt files
        for txt_file in txt_files:
            specification = txt_file.split(os.sep)[-1].split("_")[1:]
            if specification[-2] != identifier:
                continue
            if specification[0] == 'GBM':
                start_index = 0
            elif specification[0] == 'Kou':
                start_index = 2
            else: 
                continue
            count += 1
            # Open and read the file
            with open(txt_file, 'r') as file:
                content = file.read()
                metric_strings = content.split('\n')

                n_in = int(specification[1 + start_index])
                n_out = int(specification[2 + start_index])
                metric1 = metric_strings[0].split('=')[1][1:-1]
                metric1 = round(float(metric1) / 100, 6)
                metric2 = metric_strings[1].split('=')[1][1:-1]
                metric2 = round(float(metric2) / 100, 6)
                
                # Add data to dictionaries
                if n_in not in metric1_data:
                    metric1_data[n_in] = {}
                if n_in not in metric2_data:
                    metric2_data[n_in] = {}
                    
                metric1_data[n_in][n_out] = metric1
                metric2_data[n_in][n_out] = metric2

                if count == 25:
                    # Create DataFrames from the dictionaries
                    metric1_df = pd.DataFrame.from_dict(metric1_data, orient='index').sort_index(axis=0).sort_index(axis=1)
                    metric2_df = pd.DataFrame.from_dict(metric2_data, orient='index').sort_index(axis=0).sort_index(axis=1)
                    print(metric1_data)
                    print(metric1_df)
                    # Export the DataFrames to different sheets in the same Excel file
                    metric1_df.to_excel(writer, sheet_name=f'metric1_{specification[0] + identifier}', index=True)
                    metric2_df.to_excel(writer, sheet_name=f'metric2_{specification[0] + identifier}', index=True)
                    # Reset dictionaries to store metrics
                    metric1_data = {}
                    metric2_data = {}
                    count = 0